In [ ]:
import pandas as pd
import glob
import os
import warnings
import re
from datetime import datetime, timedelta

warnings.filterwarnings("ignore")

# ==========================================
# 1. 파일 경로 설정 (여기를 꼭 확인해주세요!)
# ==========================================
base_dir = os.path.join(os.getcwd(), "Gwanak_Data")

# Clean 폴더 우선 사용
clean_dir = os.path.join(base_dir, "Clean_XLSX")
if os.path.exists(clean_dir) and len(glob.glob(os.path.join(clean_dir, "*.xlsx"))) > 0:
    source_dir = clean_dir
else:
    source_dir = base_dir

# ★★★ [중요] 수작업한 정답 파일 이름 ★★★
# 파일 이름이 정확한지 꼭 확인해주세요! (확장자까지)
manual_file_name = "관악구_맛집_마스터_빈도포함_가나다순.xlsx" 
# 만약 csv라면 "관악구_맛집_마스터_빈도포함_가나다순.csv" 로 바꿔주세요.

manual_file_path = os.path.join(base_dir, manual_file_name)

# ==========================================
# 2. 정답지(수작업 리스트) 로딩 (무적 로직)
# ==========================================
print(f"📋 정답 리스트 읽기 시도: {manual_file_path}")

if not os.path.exists(manual_file_path):
    print(f"🚨 [에러] 파일을 찾을 수 없습니다! 경로와 파일명을 확인해주세요.")
    print(f"👉 찾는 경로: {manual_file_path}")
    exit()

try:
    # 1) 엑셀(.xlsx)인 경우
    if manual_file_name.endswith('.xlsx') or manual_file_name.endswith('.xls'):
        valid_df = pd.read_excel(manual_file_path)
    # 2) CSV(.csv)인 경우 (인코딩 자동 시도)
    else:
        try:
            valid_df = pd.read_csv(manual_file_path, encoding='utf-8')
        except UnicodeDecodeError:
            try:
                valid_df = pd.read_csv(manual_file_path, encoding='cp949')
            except UnicodeDecodeError:
                valid_df = pd.read_csv(manual_file_path, encoding='euc-kr')

    # 컬럼 이름 확인 및 보정
    print(f"👀 파일에 있는 컬럼들: {list(valid_df.columns)}")
    
    target_col = '식당이름'
    if target_col not in valid_df.columns:
        # '식당이름' 컬럼이 없으면, 무조건 첫 번째 컬럼을 식당 이름으로 씁니다.
        target_col = valid_df.columns[0]
        print(f"⚠️ '식당이름' 컬럼을 못 찾아서 첫 번째 컬럼인 '{target_col}'을 사용합니다.")
    
    # 정답 식당 이름 확보
    valid_names = set(valid_df[target_col].astype(str).str.strip().tolist())
    print(f"✅ 총 {len(valid_names)}개의 정답 식당 이름을 확보했습니다!")

except Exception as e:
    print(f"❌ 파일을 읽는 도중 치명적인 에러가 발생했습니다:\n{e}")
    exit()

# ==========================================
# 3. 원본 데이터 로딩 & 매칭
# ==========================================
print("데이터 복원 및 매칭 중...", end='', flush=True)

file_list = glob.glob(os.path.join(source_dir, "*.xls*"))
all_data = []

for file_path in file_list:
    # 수작업 파일이나 결과 파일은 읽지 않도록 제외
    if manual_file_name in file_path or "결과" in file_path or "통합본" in file_path: continue
    
    try:
        df = pd.read_excel(file_path)
        if '집행장소' not in df.columns:
            for idx, row in df.iterrows():
                if '집행장소' in row.values:
                    df.columns = row
                    df = df[idx+1:]
                    break
        all_data.append(df)
    except:
        pass

if not all_data:
    print("\n❌ 원본 데이터를 하나도 못 읽었습니다.")
    exit()

combined_df = pd.concat(all_data, ignore_index=True)
combined_df.columns = combined_df.columns.astype(str).str.strip()

col_place = '집행장소'
if col_place not in combined_df.columns:
    print("\n🚨 원본 데이터에서 '집행장소' 컬럼을 찾을 수 없습니다.")
    exit()

# 괄호 제거 후 임시 이름 생성
combined_df['임시장소명'] = combined_df[col_place].astype(str).str.split('(').str[0].str.strip()

# ★ 핵심: 정답 리스트에 있는 식당만 남기기 (Reverse Mapping)
matched_df = combined_df[combined_df['임시장소명'].isin(valid_names)]

print(" 완료!")
print(f"📊 매칭 결과: 원본 {len(combined_df)}건 중 {len(matched_df)}건이 정답 리스트와 일치합니다.")

# ==========================================
# 4. 상세 정보 계산
# ==========================================
col_amount = '집행금액(원)'
col_people = '대상인원수'
col_day = '집행일'
col_time = '집행시간'

# 숫자 변환
for col in [col_amount, col_people]:
    if col in matched_df.columns:
        matched_df[col] = pd.to_numeric(matched_df[col].astype(str).str.replace(',', ''), errors='coerce').fillna(0)

# 1인당 금액
if col_amount in matched_df.columns and col_people in matched_df.columns:
    matched_df['1인당금액'] = matched_df.apply(
        lambda x: x[col_amount] / x[col_people] if x[col_people] > 0 else 0, axis=1
    )
else:
    matched_df['1인당금액'] = 0

# 날짜 & 시간
if col_day in matched_df.columns:
    matched_df[col_day] = pd.to_datetime(matched_df[col_day], errors='coerce')

def extract_hour(time_val):
    try:
        time_str = str(time_val).strip()
        if ':' in time_str: return int(time_str.split(':')[0])
        elif '시' in time_str: return int(re.search(r'(\d+)시', time_str).group(1))
        else: return None
    except: return None
    
if col_time in matched_df.columns:
    matched_df['시각'] = matched_df[col_time].apply(extract_hour)
else:
    matched_df['시각'] = None

# ==========================================
# 5. 마스터 테이블 생성 (집계)
# ==========================================
print("📊 상세 정보 집계 중...")

one_year_ago = datetime.now() - timedelta(days=365)

# 조건 플래그 생성
matched_df['is_recent'] = matched_df[col_day].apply(lambda x: 1 if pd.notnull(x) and x >= one_year_ago else 0)
matched_df['is_lunch'] = matched_df['시각'].apply(lambda x: 1 if pd.notnull(x) and 11 <= x <= 14 else 0)
matched_df['is_dinner'] = matched_df['시각'].apply(lambda x: 1 if pd.notnull(x) and x >= 17 else 0)
matched_df['is_cheap'] = matched_df['1인당금액'].apply(lambda x: 1 if 10000 <= x < 20000 else 0)
matched_df['is_expensive'] = matched_df['1인당금액'].apply(lambda x: 1 if x >= 30000 else 0)

# 식당별로 묶어서 통계 내기
master_df = matched_df.groupby('임시장소명').agg(
    총방문횟수=('임시장소명', 'count'),
    평균1인당금액=('1인당금액', 'mean'),
    최근1년_방문수=('is_recent', 'sum'),
    점심_방문수=('is_lunch', 'sum'),
    저녁_방문수=('is_dinner', 'sum'),
    가성비_방문수=('is_cheap', 'sum'),
    고급_방문수=('is_expensive', 'sum')
).reset_index()

master_df['평균1인당금액'] = master_df['평균1인당금액'].round(0)
master_df.rename(columns={'임시장소명': '식당이름'}, inplace=True)

# 가나다순 정렬
master_df = master_df.sort_values(by='식당이름')

# ==========================================
# 6. 저장
# ==========================================
save_path = os.path.join(base_dir, "관악구_맛집_상세정보_통합본.xlsx")
print("💾 파일 저장 중...")

master_df.to_excel(save_path, index=False)

print(f"\n🎉 [성공] 이번엔 진짜 됐습니다! 에러 없이 완료!")
print(f"📂 파일 위치: {save_path}")

In [ ]:
# 10회 미만 방문횟수 삭제
import pandas as pd
import glob
import os
import warnings

warnings.filterwarnings("ignore")

# ==========================================
# 1. 파일 찾기 (자동 감지)
# ==========================================
base_dir = r"C:\Users\1108j\OneDrive\바탕 화면\data prep\맛집 추출_시군구\Gwanak_Data"

# Clean 폴더가 있으면 거기부터 봅니다
if os.path.exists(os.path.join(base_dir, "Clean_XLSX")):
    search_dir = os.path.join(base_dir, "Clean_XLSX")
else:
    search_dir = base_dir

print(f"📂 폴더에서 '통합본' 파일을 찾는 중... ({search_dir})")

# '통합본'이라는 단어가 들어가는 엑셀이나 CSV 파일을 다 찾습니다.
candidates = glob.glob(os.path.join(search_dir, "*통합본*.*"))

if not candidates:
    # 못 찾았으면 상위 폴더도 한 번 더 뒤집니다
    candidates = glob.glob(os.path.join(base_dir, "*통합본*.*"))

if not candidates:
    print("❌ '통합본' 파일을 못 찾겠어요! 파일 이름에 '통합본'이 있는지 확인해주세요.")
    exit()

target_file = candidates[0] # 첫 번째로 발견된 놈을 잡습니다.
print(f"👉 발견된 파일: {target_file}")

# ==========================================
# 2. 파일 읽기 (엑셀/CSV 구분)
# ==========================================
try:
    if target_file.endswith('.csv'):
        # CSV면 인코딩 여러 개 시도
        try:
            df = pd.read_csv(target_file, encoding='utf-8')
        except:
            df = pd.read_csv(target_file, encoding='cp949')
    else:
        # 엑셀이면 그냥 읽기
        df = pd.read_excel(target_file)
    
    print(f"📊 데이터 로딩 성공! (총 {len(df)}개 식당)")

except Exception as e:
    print(f"❌ 파일을 읽다가 에러가 났어요: {e}")
    exit()

# ==========================================
# 3. 10회 미만 삭제 (필터링)
# ==========================================
col_count = '총방문횟수'

if col_count not in df.columns:
    print(f"🚨 '{col_count}' 컬럼이 안 보여요. 컬럼 이름을 확인해주세요.")
    print(f"현재 컬럼 목록: {list(df.columns)}")
else:
    # 숫자 변환 (혹시 문자로 되어 있을까봐 안전장치)
    df[col_count] = pd.to_numeric(df[col_count], errors='coerce').fillna(0)
    
    # 10회 이상 필터링
    filtered_df = df[df[col_count] >= 10]
    
    print(f"✂️ 10회 미만 삭제 완료! ({len(df)}개 -> {len(filtered_df)}개)")

    # ==========================================
    # 4. 저장
    # ==========================================
    save_path = os.path.join(base_dir, "관악구_맛집_최종_10회이상.xlsx")
    
    filtered_df.to_excel(save_path, index=False)
    print(f"\n🎉 [성공] 저장 완료!\n📂 파일 위치: {save_path}")